In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

# Parámetros para Cuckoo Search
import random

# Ignorar warnings
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# Carga de datos
df = pd.read_csv('datos.csv', encoding='latin1')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166684 entries, 0 to 166683
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   ANIO                166684 non-null  int64 
 1   PROVINCIA           166684 non-null  object
 2   CANTON              166684 non-null  object
 3   FECHA               166684 non-null  object
 4   HORA                166684 non-null  object
 5   TIPO_DE_SINIESTRO   166684 non-null  object
 6   TIPO_DE_VEHICULO_1  166684 non-null  object
 7   EDAD_1              166684 non-null  int64 
 8   SEXO_1              166684 non-null  object
 9   CONDICION_1         166684 non-null  object
 10  CASCO_1             166684 non-null  object
 11  CINTURON_1          166684 non-null  object
dtypes: int64(2), object(10)
memory usage: 15.3+ MB


In [ ]:
# Convertir la columna 'FECHA' a formato de fecha
df['FECHA'] = pd.to_datetime(df['FECHA'], format='%d/%m/%Y')

# Convertir la columna 'HORA' a formato de tiempo
df['HORA'] = pd.to_datetime(df['HORA'], format='%H:%M:%S').dt.hour

In [ ]:
# Codificar las variables categóricas
label_encoder = LabelEncoder()
df['PROVINCIA'] = label_encoder.fit_transform(df['PROVINCIA'])
df['CANTON'] = label_encoder.fit_transform(df['CANTON'])
df['TIPO_DE_SINIESTRO'] = label_encoder.fit_transform(df['TIPO_DE_SINIESTRO'])
df['TIPO_DE_VEHICULO_1'] = label_encoder.fit_transform(df['TIPO_DE_VEHICULO_1'])
df['SEXO_1'] = label_encoder.fit_transform(df['SEXO_1'])
df['CONDICION_1'] = label_encoder.fit_transform(df['CONDICION_1'])
df['CASCO_1'] = label_encoder.fit_transform(df['CASCO_1'])
df['CINTURON_1'] = label_encoder.fit_transform(df['CINTURON_1'])

In [ ]:
# Definir las características (X) y la variable objetivo (y)
X = df.drop(['CONDICION_1', 'FECHA'], axis=1)
y = df['CONDICION_1']

# Escalar las características
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def create_model(optimizer='adam', init='uniform'):
    model = Sequential()
    model.add(Dense(12, input_dim=X_train.shape[1], kernel_initializer=init, activation='relu'))
    model.add(Dense(8, kernel_initializer=init, activation='relu'))
    model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [ ]:
import random

def cuckoo_search(nests, pa=0.25, beta=1.5, n_iterations=100):
    def levy_flight(Lambda):
        sigma1 = (np.math.gamma(1 + Lambda) * np.sin(np.pi * Lambda / 2) /
                  (np.math.gamma((1 + Lambda) / 2) * Lambda * 2 ** ((Lambda - 1) / 2))) ** (1 / Lambda)
        sigma2 = 1
        u = np.random.normal(0, sigma1, size=1)
        v = np.random.normal(0, sigma2, size=1)
        step = u / abs(v) ** (1 / Lambda)
        return step

    def get_fitness(nests):
        fitness = []
        for nest in nests:
            model = create_model(optimizer=nest['optimizer'], init=nest['init'])
            history = model.fit(X_train, y_train, epochs=10, batch_size=10, verbose=0)
            _, accuracy = model.evaluate(X_test, y_test, verbose=0)
            fitness.append(accuracy)
        return fitness

    num_nests = len(nests)
    fitness = get_fitness(nests)
    best_nest = nests[np.argmax(fitness)]

    for _ in range(n_iterations):
        new_nests = []
        for nest in nests:
            step_size = levy_flight(beta)
            new_nest = {key: nest[key] for key in nest}
            new_nest['optimizer'] = np.random.choice(['adam', 'sgd', 'rmsprop', 'adagrad'])
            new_nest['init'] = np.random.choice(['uniform', 'normal', 'he_normal'])
            new_nests.append(new_nest)

        new_fitness = get_fitness(new_nests)
        for i in range(num_nests):
            if new_fitness[i] > fitness[i]:
                nests[i] = new_nests[i]
                fitness[i] = new_fitness[i]

        best_nest = nests[np.argmax(fitness)]
        # Abandono aleatorio
        for nest in nests:
            if random.uniform(0, 1) < pa:
                nest['optimizer'] = np.random.choice(['adam', 'sgd', 'rmsprop', 'adagrad'])
                nest['init'] = np.random.choice(['uniform', 'normal', 'he_normal'])

    return best_nest

# Inicializar nidos
nests = [{'optimizer': 'adam', 'init': 'uniform'} for _ in range(10)]

# Ejecutar la búsqueda Cuckoo
best_params = cuckoo_search(nests)

# Crear el modelo con los mejores parámetros encontrados
model = create_model(optimizer=best_params['optimizer'], init=best_params['init'])
